## bw77
- 테스트 메서드를 실행하기 전에 테스트 환경을 구축해야 하는 경우가 자주 있는데 이런 테스트 과정을 테스트 하네스(test harness) 라고 한다.
- 단위테스트(unit test)와 통합 테스트(integration test)가 필요한 이유는 모듈이 제대로 동작하는지 검증하기 위해서이다.

In [ ]:
#!/usr/bin/env python3.8
from pathlib import Path
from tempfile import TemporaryDirectory
from unittest import TestCase, main

class EnvironmentTest(TestCase):
    def setUp(self):   #  테스트 메서드를 실행하기 전에 호출
        self.test_dir = TemporaryDirectory()
        self.test_path = Path(self.test_dir.name)

    def tearDown(self):  # 테스트 메서드를 실행한 다음 호출
        self.test_dir.cleanup()

    def test_modify_file(self):   # 테스트 메서드
        with open(self.test_path / 'data.bin', 'w') as f:
            pass

if __name__ == '__main__':
    main()

# setUp, tearDown 메서드를 활용하면 각 테스트를 서로 격리된 상태에서 실행할 수 있다.

# 출력
# >>>
# ----------------------------------------------------------------------
# Ran 1 test in 0.006s
#
# OK

- 테스트 환경을 구축할 때 계산비용이 비싸거나, 오랜시간이 소요되는 경우에는 setUp과 tearDown 메서드에서 테스트를 준비하고 정리하는 과정이 비실용적이다
  - Ex) DB 프로세스를 시작하고 DB가 모든 인덱스를 메모리에 읽어올 때까지 기다려야 할 경우 등
- 이런 상황을 해결하기 위해 unittest 모듈은 테스트 하네스 초기화를 지원한다.
  - 이 기능을 활용하면 비싼 자원을 한번만 초기화하고, 초기화를 반복하지 않고 모든 TestCase 클래스와 테스트 메서드를 실행 할 수 있다. 
  - 나중에 모듈 안의 모든 테스트가 끝나면 테스트 하네스를 단 한번만 정리하면 된다.
  - TestCase 클래스가 들어있는 모듈 안에 setUpModule과 tearDownModule 메서드를 정의해 위 동작을 사용한다.

In [ ]:
#!/usr/bin/env python3.8
# integration_test.py
from unittest import TestCase, main

def setUpModule():
    print('* 모듈 설정')

def tearDownModule():
    print('* 모듈 정리')

class IntegrationTest(TestCase):
    def setUp(self):
        print('* 테스트 설정')

    def tearDown(self):
        print('* 테스트 정리')

    def test_end_to_end1(self):
        print('* 테스트 1')

    def test_end_to_end2(self):
        print('* 테스트 2')

if __name__ == '__main__':
    main()

# >>>
# * 모듈 설정
# * 테스트 설정
# * 테스트 1
# * 테스트 정리
# .* 테스트 설정
# * 테스트 2
# * 테스트 정리
# .* 모듈 정리

# ----------------------------------------------------------------------
# Ran 2 tests in 0.002s

# OK

- mock
  - unittest.mock 모듈
  - 단위테스트를 작성하다 보면 데이터베이스 또는 외부 API에 의존하는 코드 테스트를 해야 할 일이 필연적으로 생기기 마련
  - 테스트 환경에서는 운영 환경과 대비하여 DB와 연동하거나 외부 API를 호출하기 어려운 경우가 많음
  - 외부에 의존하는 부분은 임의의 가짜로 대체하는 기법이 사용되는데 이를 Mocking 이라고 함
  - 즉, Mocking은 외부 서비스에 의존하지 않고 독립저긍로 실행이 가능한 unittest를 작성하기 위해 사용되는 테스팅 기법임
  - [파이썬 테스트 모킹](https://www.daleseo.com/python-unittest-mock/) 참고